In [1]:
import wave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import math
import librosa
import json
import wave
import sys
import pickle
import sklearn

import urllib.request

import librosa.display
import scipy, matplotlib.pyplot as plt, IPython.display as ipd

from scipy.io import wavfile

import librosa.display
import soundfile as sf

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import os
import matplotlib.style as ms
from tqdm import tqdm
import IPython.display
import random
ms.use('seaborn-muted')
%matplotlib inline

In [3]:
audio_data = "Audio/"
audio_preproccess = 'data/Audio_pre/'
annotations_data_by_emotions = "Annotations_by_emotions/"

df_angry = pd.read_csv(annotations_data_by_emotions + 'data_Angry.csv', index_col='ID')
df_disgusted = pd.read_csv(annotations_data_by_emotions + 'data_Disgusted.csv', index_col='ID')
df_happy = pd.read_csv(annotations_data_by_emotions + 'data_Happy.csv', index_col='ID')
df_neutral = pd.read_csv(annotations_data_by_emotions + 'data_Neutral.csv', index_col='ID')
df_sad = pd.read_csv(annotations_data_by_emotions + 'data_Sad.csv', index_col='ID')
df_scared = pd.read_csv(annotations_data_by_emotions + 'data_Scared.csv', index_col='ID')
df_surprised = pd.read_csv(annotations_data_by_emotions + 'data_Surprised.csv', index_col='ID')

In [ ]:
df_angry['Emotion'] = 'ang'
df_disgusted['Emotion'] = 'dis' 
df_happy['Emotion'] = 'hap' 
df_neutral['Emotion'] = 'neu' 
df_sad['Emotion'] = 'sad'
df_scared['Emotion'] = 'sca' 
df_surprised['Emotion'] = 'sur'

In [ ]:
df_complete = pd.concat([df_happy, df_angry, df_disgusted, df_neutral, df_sad, df_scared, df_surprised])
df_complete.File += '_mic.wav'

In [ ]:
df_complete.head()

In [ ]:
df_complete.to_csv('df_complete.csv', index=False)

In [2]:
labels_df = pd.read_csv('df_complete.csv')

In [ ]:
sr = 44100
audio_vectors = {}
metadata_vectors = {}
for sess in [1]:  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = 'Audio/'
    orig_wav_files = os.listdir(wav_file_path)
    for orig_wav_file in tqdm(orig_wav_files):
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            orig_wav_file, file_format = orig_wav_file.split('.')
            for index, row in labels_df[labels_df['File'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion = row['Start'], row['End'], row['File'], row['Emotion']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                audio_vectors["{}_".format(index) + truncated_wav_file_name] = (truncated_wav_vector, emotion)
#                 metadata_vectors["{}_".format(index) + truncated_wav_file_name] = emotion
        except KeyboardInterrupt:
            break
        except:
            print('An exception occured for {}'.format(orig_wav_file))
#     with open('audio_vectors/audio_vectors_{}.pkl'.format(sess), 'wb') as f:
#         pickle.dump(audio_vectors, f)

In [ ]:
len(audio_vectors)

In [ ]:
path_preproccess = "Audio_preprocess/"
# sf.write('stereo_file.wav', np.random.randn(10, 2), 44100, 'PCM_24')
files = []
emotions = []
for i in tqdm(audio_vectors):
    files.append(path_preproccess + i)
    emotions.append(audio_vectors[i][1])
    sf.write(path_preproccess + i, audio_vectors[i][0], 44100, 'PCM_24')
preprop_data = {"File": files, "Emotion" : emotions}
df_prep = pd.DataFrame.from_dict(preprop_data)
df_prep.to_csv('df_prep.csv', index=False)

In [ ]:
ipd.Audio('Audio_preprocess/3281_22dec_D62_1_mic.wav')

In [9]:
labels_df.to_numpy()

array([['10dec_D11_1_mic.wav', 104.75, 116.5, 'hap'],
       ['10dec_D11_1_mic.wav', 199.3, 210.22, 'hap'],
       ['10dec_D42_1_mic.wav', 27.153, 29.44, 'hap'],
       ...,
       ['22dec_K42_1_mic.wav', 25.02, 30.32, 'sur'],
       ['22dec_K43_2_mic.wav', 6.72, 20.27, 'sur'],
       ['22dec_K44_2_mic.wav', 18.227, 23.67, 'sur']], dtype=object)